<h1><center>COVID-19. Детекция Атипичной пневмонии по рентгеновским снимкам</center></h1>

<center><img src = "https://ichef.bbci.co.uk/news/800/cpsprodpb/1EA6/production/_110764870_coronavirusmedium.jpg.webp" width = "750" height = "500"/></center>

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>[Трнировка Модели]</center></h2>

### Ссылки

[Detectron_offical](https://github.com/facebookresearch/detectron2)

[VinBigData detectron2](https://www.kaggle.com/corochann/vinbigdata-detectron2-train)

В качестве фреймворка я выбрал Detectron2 по ряду причин:

    - Субъективно нравится
    - Понятный мануал
    - Неплохие результаты ( пробовал его в соревновании на Кагле)
    - Возможность гибкой настройки обучения

**DETECTRON2**

Detectron2 - это система нового поколения от Facebook AI Research, реализующая самые современные алгоритмы обнаружения объектов.

# Импорт Библиотек

In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
from tqdm import tqdm_notebook as tqdm # progress bar
import matplotlib.pyplot as plt

import os, json, cv2, random
import skimage.io as io
import copy
import pickle
from pathlib import Path
from typing import Optional

import torch

# Albumenatations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings

import config

In [2]:
# detectron2
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils


from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

setup_logger()

<Logger detectron2 (DEBUG)>

In [3]:
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('ggplot')

# Загрузка данных

In [4]:
imdir = f'{config.DATA_ROOT_RESIZED}/img_sz_640'
# Загрузим CSV полученный в ноутбуке с EDA
train_df = pd.read_csv(f'{config.DATA_ROOT}/train_image_df.csv')
print(f'Number of studies: {len(train_df)}')

Number of studies: 6334


## Configs

In [5]:
classes = [
    "atypical",
    "indeterminate",
    "negative",
    "typical"
]

debug=False
split_mode="valid20" # Or  valid20 all_train

category_name_to_id = {
    class_name: index for index, class_name in enumerate(classes)
}

print(category_name_to_id)

{'atypical': 0, 'indeterminate': 1, 'negative': 2, 'typical': 3}


<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>[Подготовка данных]</center></h2>

Для определения кстомного датасета, нам нужно создать **список словарей** (dataset_dicts) где каждый словарь содержит:

 - file_name: имя файла изображения
 - image_id: id изображения, (он же индекс).
 - height: высота изображения.
 - width: Ширина изображения.
 - annotation: основные аннотационные данные для обнаружения объектов, которые содержат следующее:
   - bbox: bounding box pixel location имеющий shape n_boxes,4
   - bbox_mode: Тут тспользуется BoxMode.XYXY_ABS. Абсолютное значение (x_min, y_min, x_max, y_max) аннотаций используется в bbox
   - category_id: class label id for each bounding box, with shape (n_boxes,)


*build_COVID19_data_dicts* для подготовки тренировочных даных
*build_COVID19_data_dicts_test* для подготовки тестовых даных

In [6]:
df_meta = pd.read_csv(f'{config.DATA_ROOT_RESIZED}/img_sz_640/meta_sz_640.csv')

In [7]:
from COVID19_dict import build_COVID19_data_dicts

In [9]:
build_COVID19_data_dicts(imdir,train_df,debug=debug,data_type="train")

Load from cache dataset_dicts_cache_train.pkl


[{'file_name': './dataset/SIIM-COVID19-Resized/img_sz_640/train/d8ba599611e5.jpg',
  'image_id': 'd8ba599611e5',
  'height': 640,
  'width': 640,
  'annotations': []},
 {'file_name': './dataset/SIIM-COVID19-Resized/img_sz_640/train/29b23a11d1e4.jpg',
  'image_id': '29b23a11d1e4',
  'height': 640,
  'width': 640,
  'annotations': [{'bbox': [328.2129428571428,
     108.80000000000001,
     545.0109639097744,
     536.5333339449542],
    'bbox_mode': <BoxMode.XYXY_ABS: 0>,
    'category_id': 3}]},
 {'file_name': './dataset/SIIM-COVID19-Resized/img_sz_640/train/8174f49500a5.jpg',
  'image_id': '8174f49500a5',
  'height': 640,
  'width': 640,
  'annotations': []},
 {'file_name': './dataset/SIIM-COVID19-Resized/img_sz_640/train/d54f6204b044.jpg',
  'image_id': 'd54f6204b044',
  'height': 640,
  'width': 640,
  'annotations': []},
 {'file_name': './dataset/SIIM-COVID19-Resized/img_sz_640/train/d51cadde8626.jpg',
  'image_id': 'd51cadde8626',
  'height': 640,
  'width': 640,
  'annotations': [